In [1]:
##########
# Import libraries
##########
import pandas as pd
import numpy as np

# Neighborhood Data

In [2]:
# Read in csv files
demographics_df = pd.read_csv(r'community_demographic_cleaned.csv')
covid_df = pd.read_csv(r'community_todayUpdated.csv')

In [3]:
# Get only race/ethnicity data from demographics_df
city_race_percent_df = demographics_df[["Neighborhood", "ASIAN.POPULATION", "BLACK.POPULATION",
                                "LATINO.POPULATION", "WHITE.POPULATION", "NON.WHITE.POPULATION",
                               "POPULATION.TOTAL"]]
city_race_percent_df = city_race_percent_df.sort_values(by='Neighborhood', ignore_index=True)

city_race_percent_df

,Neighborhood,ASIAN.POPULATION,BLACK.POPULATION,LATINO.POPULATION,WHITE.POPULATION,NON.WHITE.POPULATION,POPULATION.TOTAL
0,Acton,0.015,0.008,0.124,0.817,0.183,6522
1,Adams-Normandie,0.052,0.248,0.622,0.056,0.944,17596
2,Agoura Hills,0.062,0.008,0.070,0.828,0.172,20324
3,Agua Dulce,0.002,0.002,0.109,0.839,0.161,3130
4,Alhambra,0.472,0.014,0.355,0.138,0.862,85961
...,...,...,...,...,...,...,...
260,Willowbrook,0.006,0.443,0.530,0.009,0.991,34336
261,Wilmington,0.027,0.026,0.866,0.064,0.936,51517
262,Windsor Square,0.416,0.043,0.148,0.377,0.623,5642
263,Winnetka,0.155,0.043,0.406,0.361,0.639,46943


In [4]:
##########
# Calculate the total population of each race in each city
# (# people of given race) = (% of city that is given race) * (# people in city)
##########

# This stores column name and corresponding number from city_race_percent_df
column_codes =  [('ASIAN', 1), ('BLACK', 2), ('LATINO', 3), ('WHITE', 4), ('NON.WHITE', 5)]

# Initialize our empty dictionary
city_race_info = {}
city_race_info['CITY'] = []
for race, j in column_codes:
    city_race_info[race] = []

# Fill in dictionary
for i, row in city_race_percent_df.iterrows():
    # Total population of a given city
    total_pop = row[6]
    # Store name of city in city_race_info
    city_name = row[0]
    city_race_info['CITY'].append(city_name)
    # Loop through all races in city_race_percent_df
    for race, j in column_codes:
        city_race_info[race].append(row[j] * total_pop)
            
# Convert to DataFrame and sort by alphabetical city-name
city_race_df = pd.DataFrame(city_race_info)
city_race_df = city_race_df.sort_values(by='CITY', ignore_index=True)

city_race_df

,CITY,ASIAN,BLACK,LATINO,WHITE,NON.WHITE
0,Acton,97.830,52.176,808.728,5328.474,1193.526
1,Adams-Normandie,914.992,4363.808,10944.712,985.376,16610.624
2,Agoura Hills,1260.088,162.592,1422.680,16828.272,3495.728
3,Agua Dulce,6.260,6.260,341.170,2626.070,503.930
4,Alhambra,40573.592,1203.454,30516.155,11862.618,74098.382
...,...,...,...,...,...,...
260,Willowbrook,206.016,15210.848,18198.080,309.024,34026.976
261,Wilmington,1390.959,1339.442,44613.722,3297.088,48219.912
262,Windsor Square,2347.072,242.606,835.016,2127.034,3514.966
263,Winnetka,7276.165,2018.549,19058.858,16946.423,29996.577


In [5]:
# Get only city name, city population, and cumulative case numbers from covid_df
city_covid_df = covid_df[["place_ID", "population", "cumulative_cases_today"]]
city_covid_df = city_covid_df.rename(columns={"place_ID":"CITY", 
                                              "population":"POPULATION", 
                                              "cumulative_cases_today":"POSITIVES"})
city_covid_df = city_covid_df.sort_values(by='CITY', ignore_index=True)

city_covid_df

,CITY,POPULATION,POSITIVES
0,Adams Normandie,8202,365
1,Agoura Hills,20883,271
2,Alhambra,86724,1738
3,Alsace,12445,504
4,Angeles National Forest,40,2
...,...,...,...
218,Wholesale District,36129,2898
219,Wilmington,56487,2322
220,Wilshire Center,50170,1497
221,Winnetka,51786,1973


In [6]:
##########
# Concatenate all data for each city into one dataframe
##########

# This stores column name and corresponding number from city_race_percent_df
column_codes =  [('ASIAN', 1), ('BLACK', 2), ('LATINO', 3), ('WHITE', 4), ('NON.WHITE', 5)]

# Initialize empty dictionary
city_info = {}
city_info['CITY'] = []
for race, j in column_codes:
    city_info[race] = []
city_info['POSITIVES'] = []
city_info['POPULATION'] = []

# Fill in dictionary
for i, city_race_row in city_race_df.iterrows():
    # Get city name
    city = city_race_row[0]
    
    # Find city in city covid data
    city_covid_data = city_covid_df.loc[city_covid_df['CITY'] == city]
    # if city does not exist in city covid data, then continue to next city
    if city_covid_data.empty:
        continue
    # if city exists in both city demographic data and city covid data, then add it to city info 
    else:
        # add demographic data
        city_info['CITY'].append(city)
        for race, j in column_codes:
            city_info[race].append(city_race_row[j])

        # add covid data
        index = city_covid_df.loc[city_covid_df['CITY'] == city].index.to_numpy().item()
        city_info['POPULATION'].append(city_covid_data['POPULATION'].loc[index])
        city_info['POSITIVES'].append(city_covid_data['POSITIVES'].loc[index])

city_df = pd.DataFrame(city_info)
city_df

,CITY,ASIAN,BLACK,LATINO,WHITE,NON.WHITE,POSITIVES,POPULATION
0,Agoura Hills,1260.088,162.592,1422.680,16828.272,3495.728,271,20883
1,Alhambra,40573.592,1203.454,30516.155,11862.618,74098.382,1738,86724
2,Arcadia,24039.754,423.608,5242.149,21339.253,31611.747,716,57754
3,Arleta,3417.480,683.496,22275.756,4100.976,26967.024,2197,34370
4,Artesia,4520.880,491.400,6273.540,4275.180,12104.820,516,16795
...,...,...,...,...,...,...,...,...
162,Westwood,11068.596,958.320,3354.120,30139.164,17776.836,715,54109
163,Whittier,3185.844,754.542,46781.604,31103.898,52734.102,3285,87432
164,Wilmington,1390.959,1339.442,44613.722,3297.088,48219.912,2322,56487
165,Winnetka,7276.165,2018.549,19058.858,16946.423,29996.577,1973,51786


# Countywide Data

In [7]:
#countyDF = pd.read_csv(r'PATHNAME')

In [8]:
county_race_covid = {'Race/Ethnicity':['American Indian/Alaska Native','Asian', 
                                       'Black','Hispanic/Latino', 
                                      'Native Hawaiian/Pacific Islander','White'],
                    'Cases':[316, 10473, 10113, 129999, 967, 28990]}
county_race_covid_df = pd.DataFrame(county_race_covid)

In [9]:
##########
# Clean up county_race_covid_df
##########
county_race_covid_df.drop(index=0, inplace=True)
county_race_covid_df.drop(index=4, inplace=True)

non_white = 0
for i, row in county_race_covid_df.iterrows():
    upper = row[0].upper()
    if upper == 'HISPANIC/LATINO':
        upper = 'LATINO'
    county_race_covid_df.at[i, 'Race/Ethnicity'] = upper
    if upper != 'WHITE':
        non_white += county_race_covid_df.at[i, 'Cases']

non_white_row = {'Race/Ethnicity':'NON.WHITE', 
                 'Cases' : non_white}
county_race_covid_df = county_race_covid_df.append(non_white_row, ignore_index=True)

county_race_covid_df

,Race/Ethnicity,Cases
0,ASIAN,10473
1,BLACK,10113
2,LATINO,129999
3,WHITE,28990
4,NON.WHITE,150585


In [10]:
##########
# Demographics data for LA County from U.S. Census Bureau
# https://www.census.gov/quickfacts/losangelescountycalifornia
##########
county_race = {'Race/Ethnicity':['American Indian/Alaska Native',
                                    'Asian', 'Black','Hispanic/Latino', 
                                    'Native Hawaiian/Pacific Islander', 'White',],
                'Percent':[0.004, 0.154, 0.09, 0.486, 0.004, 0.261],
              }
total_pop = 10039107
pop_num = np.zeros_like(county_race['Percent'])
for i in np.arange(len(county_race['Percent'])):
    pop_num[i] = int(county_race['Percent'][i] * total_pop)
county_race['Population'] = pop_num

county_race_df = pd.DataFrame(county_race)
county_race_df.drop(columns='Percent', inplace=True)

In [11]:
##########
# Clean up county_race_df
##########
county_race_df.drop(index=0, inplace=True)
county_race_df.drop(index=4, inplace=True)

non_white = 0
for i, row in county_race_df.iterrows():
    upper = row[0].upper()
    if upper == 'HISPANIC/LATINO':
        upper = 'LATINO'
    county_race_df.at[i, 'Race/Ethnicity'] = upper
    if upper != 'WHITE':
        non_white += county_race_df.at[i, 'Population']

non_white_row = {'Race/Ethnicity':'NON.WHITE', 
                 'Population' : non_white}
county_race_df = county_race_df.append(non_white_row, ignore_index=True)

county_race_df

,Race/Ethnicity,Population
0,ASIAN,1546022.0
1,BLACK,903519.0
2,LATINO,4879006.0
3,WHITE,2620206.0
4,NON.WHITE,7328547.0


In [12]:
##########
# Merge county_race_df with county_race_covid_df
##########
county_info = {'RACE':[], 'POPULATION':[], 'POSITIVES':[]}
for i, row in county_race_df.iterrows():
    county_info['RACE'].append(row[0])
    county_info['POPULATION'].append(row[1])
    county_info['POSITIVES'].append(county_race_covid_df.at[i, 'Cases'])
        
county_df = pd.DataFrame(county_info)
county_df

,RACE,POPULATION,POSITIVES
0,ASIAN,1546022.0,10473
1,BLACK,903519.0,10113
2,LATINO,4879006.0,129999
3,WHITE,2620206.0,28990
4,NON.WHITE,7328547.0,150585


# Data Imputation for Neighborhoods

In [13]:
'''
PROPORTION

# interpolated positive cases per race in city      # positive cases per race in county
-------------------------------                  =   ------------------------------------
# people of race in city                            # people of race in county

SCALING

# scaled cases per race in city =    # interpolated positive cases per race in city
                                      ----------------------------------------        x total acutual cases
                                     total interpolated cases for all races in city
                                            
PERCENTAGE

% of total positives from given race = # true positive cases per race in city
                                        --------------------------------------
                                        total population of given race in city

'''

'\nPROPORTION\n\n# interpolated positive cases per race in city      # positive cases per race in county\n-------------------------------                  =   ------------------------------------\n# people of race in city                            # people of race in county\n\nSCALING\n\n# scaled cases per race in city =    # interpolated positive cases per race in city\n                                      ----------------------------------------        x total acutual cases\n                                     total interpolated cases for all races in city\n                                            \nPERCENTAGE\n\n% of total positives from given race = # true positive cases per race in city\n                                        --------------------------------------\n                                        total population of given race in city\n\n'

In [16]:
# empty dataframe that will store percentage of each city's positive cases that come from each race
city_positive_race_df = pd.DataFrame(columns=['CITY', 'ASIAN', 'BLACK', 'LATINO', 'WHITE', 'NON.WHITE',
                                             'TOTAL.POP', 'TOTAL.CASES'])
# this stores the column names and corresponding number in city_df
column_codes = [('ASIAN', 1), ('BLACK', 2), ('LATINO', 3), ('WHITE', 4), ('NON.WHITE', 5)]

# iterate through all the rows in city_race_df
for i, city_row in city_df.iterrows():
    # empty dictionary to story the data from a given city
    city_info = {}
    
    # get neighborhood name from city_race_df and store it in city_info
    city_name = city_row[0]
    city_info['CITY'] = city_name
    
    total_interpolated = 0
    raw_race = {}
    # calculate raw interpolation
    for race, j in column_codes:        
        # get population of given race in the city
        pop_city_race = city_row[j]
    
        county_race_data = county_df.loc[county_df['RACE'] == race]
        # get population of given race in the county
        pop_county_race = county_race_data['POPULATION'].loc[j-1]
        # get number of positive cases in the county for given race
        num_county_positive_race = county_race_data['POSITIVES'].loc[j-1]

        #calculate number of positives for given race in city 
        raw_race[race] = int((num_county_positive_race / pop_county_race) * pop_city_race)
        total_interpolated += raw_race[race]
    
    # scale number of cases by total cases in city and divide by race population to get percent
    total_cases_in_city = city_row[6]
    for race, j in column_codes:
        # scale number of positives by number of total positive cases per city
        scaled_num_positive = int((raw_race[race] / total_interpolated) * total_cases_in_city)
        
        # get population of given race in the city
        pop_city_race = city_row[j]
        print(pop_city_race)
        # divide scaled number by population of race in city to get final percentage
        if pop_city_race != 0:
            city_info[race] = scaled_num_positive / pop_city_race
        else:
            city_info[race] = 0.0
    
    city_info['TOTAL.CASES'] = total_cases_in_city
    city_info['TOTAL.POP'] = city_row[7]
    
    # append city_info to city_positive_race_df
    city_positive_race_df = city_positive_race_df.append(city_info, ignore_index=True)
        
city_positive_race_df

1260.088
162.592
1422.68
16828.272
3495.7279999999996
40573.592000000004
1203.454
30516.155
11862.618
74098.382
24039.753999999997
423.608
5242.149
21339.252999999997
31611.747
3417.48
683.4960000000001
22275.756
4100.976000000001
26967.024
4520.88
491.4
6273.539999999999
4275.18
12104.820000000002
2932.9359999999997
208.432
7637.544
3305.136
11582.864
47.714999999999996
9.543000000000001
1456.898
1577.776
1603.224
2485.504
1509.056
28716.448
10474.624
33909.376000000004
8938.854000000001
1060.542
59617.611000000004
5605.722000000001
70147.27799999999
440.004
293.336
33256.969000000005
2163.353
34503.647
220.27
264.324
41278.598000000005
2070.538
41983.462
7355.728999999999
8957.967
31170.811999999998
22431.332
50397.668000000005
434.08
184.484
368.968
9495.5
1356.5
2571.004
473.60599999999994
1623.7920000000001
27672.122
6156.878
443.84
255.36
370.88
4864.0
1216.0
2226.1440000000002
834.8040000000001
87190.64
1855.1200000000001
90900.88
237.11599999999999
12.68
150.89200000000002
838.

1545.9049999999997
2036.0700000000002
8181.985
24206.61
13498.39
247.392
8782.416
11235.720000000001
206.16
20409.84
498.124
17751.328
25585.46
633.9759999999999
44650.024
186.328
10480.95
12134.611
256.201
23034.799000000003
0.0
0.0
87.044
1.9740000000000002
92.02600000000001
16862.248000000003
1140.152
5730.764
5550.74
24453.26
73.63
13658.365
22678.04
184.07500000000002
36630.925
369.988
8183.264
12231.368
522.336
21241.664
23810.711
6188.687000000001
47726.315
24544.962
80348.03799999999
4385.982
970.35
4269.54
27519.126000000004
11294.874000000002
1428.64
1000.0479999999999
3214.44
29001.392000000003
6714.608
1443.126
303.81600000000003
670.927
9709.453
2949.547
3984.0
6889.0
6847.5
21704.5
19795.5
17133.435
4049.721
76217.82600000002
4672.755
99166.245
537.106
43.315
389.835
7406.865
1256.135
11068.596000000001
958.32
3354.1200000000003
30139.164
17776.836
3185.844
754.5420000000001
46781.60399999999
31103.898
52734.102
1390.959
1339.442
44613.722
3297.088
48219.912
7276.165
2018

,CITY,ASIAN,BLACK,LATINO,WHITE,NON.WHITE,TOTAL.POP,TOTAL.CASES
0,Agoura Hills,0.005555,0.000000,0.023196,0.009864,0.018022,20883,271
1,Alhambra,0.004239,0.006648,0.016811,0.006912,0.012956,86724,1738
2,Arcadia,0.004035,0.004721,0.015833,0.006608,0.012337,57754,716
3,Arleta,0.011997,0.017557,0.047855,0.019508,0.036934,34370,2197
4,Artesia,0.006857,0.010175,0.027576,0.011228,0.021231,16795,516
...,...,...,...,...,...,...,...,...
162,Westwood,0.005421,0.008348,0.021764,0.009058,0.016820,54109,715
163,Whittier,0.007847,0.011928,0.032363,0.013439,0.024955,87432,3285
164,Wilmington,0.006470,0.010452,0.027637,0.011222,0.021298,56487,2322
165,Winnetka,0.009620,0.015358,0.037988,0.015756,0.029337,51786,1973


In [17]:
city_positive_race_df.to_csv('race-ethnicity_interpolated.csv')